<a href="https://colab.research.google.com/github/sofiyaefimova302-png/Efimova-Sophya/blob/main/%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [1]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 24.1 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [2]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Кроме вышеуказанного датасета доступны следующие датасеты: glove-wiki-gigaword-300, glove-twitter-25, word2vec-ruscorpora-300, glove-twitter-200, __testing_word2vec-matrix-synopsis,  fasttext-wiki-news-subwords-300, conceptnet-numberbatch-17-06-300, word2vec-google-news-300, glove-wiki-gigaword-50, glove-wiki-gigaword-200, glove-twitter-50, glove-twitter-100

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

1)fasttext-wiki-news-subwords-300

Источник данных: Wikipedia + News articles

Примерный объем: 1 млн. слов

Применяется в работе с опечатками и редкими словами, с морфологически богатыми языками и для обработки слов вне пределов словаря.

2)word2vec-ruscorpora-300

Источник данных: корпус русских текстов

Примерный объем: несколько миллиардов слов

Применяется для научного анализа семантики/грамматики/словоупотребления конкретно в русском языке. База для обучения, к примеру, русскоязычных ботов.

3)word2vec-google-news-300

Источник данных: Google News

Примерный объем: 3 млн. слов

Применяется для решения общих задач NLP на английском языке, классификации текстов/анализа тональности и составления поисковых систем и рекомендаций.

**Базовые операции с векторами**

In [3]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [4]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [15]:
!pip install gensim

import gensim
import gensim.downloader as api
model = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [17]:
word_u = input()
# функция ищет 10 слов, похожих на то слово, которое было введено, и выводит нужный результат
similar_words = w2v_model.most_similar(word_u, topn=10)
print(f"Слова, похожие на {word_u}:")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

avatar
Слова, похожие на avatar:
  avatars: 0.6016
  vishnu: 0.5783
  incarnation: 0.5580
  blockbuster: 0.5546
  shrek: 0.5517
  3-d: 0.5308
  kalki: 0.5014
  varaha: 0.5014
  undead: 0.4967
  vamana: 0.4926


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [5]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [18]:
#задаю параметры модели Word2Vec
model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)
print("Word2Vec модель успешно обучена :)")

Word2Vec модель успешно обучена :)


In [19]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [20]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [21]:
# Нахожу слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Нахожу слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [22]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)
print("FastText модель успешно обучена :)")

FastText модель успешно обучена :)


6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [23]:
# Нахожу слова, похожие на "варить"
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

# Нахожу слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Нахожу слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711
Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [39]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваю для разных слов
compare_models('мосг')
compare_models('скаворода')
compare_models('кащтан')

#Сравнительный анализ показал, что только FastText нашёл слово с опечаткой, но и то, только в одном случае, со словом "сковорода"
#Word2Vec же не справился вовсе


Сравнение для слова: 'мосг'
  Word2Vec: слово не найдено
  FastText: ['пирог', 'мясо']

Сравнение для слова: 'скаворода'
  Word2Vec: слово не найдено
  FastText: ['сковорода', 'горшок']

Сравнение для слова: 'кащтан'
  Word2Vec: слово не найдено
  FastText: ['лимон', 'тост']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [40]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [41]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [42]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [43]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [45]:
def compare_documents(doc2_id, doc4_id):
    similarity = doc_model.dv.similarity(f"doc_{doc2_id}", f"doc_{doc4_id}")
    print(f"Схожесть doc_{doc2_id} и doc_{doc4_id}: {similarity:.4f}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")
    print(f"  doc_{doc4_id}: {documents[doc4_id]}")

compare_documents(2, 4)


Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [46]:
compare_documents(1, 0)
compare_documents(1, 2)
compare_documents(1, 3)
compare_documents(1, 4)

#Анализ показал, что самым похожим документом на doc_1 является doc_0 с результатом 0.2735

Схожесть doc_1 и doc_0: 0.2735
  doc_1: deep learning uses neural networks
  doc_0: machine learning is interesting
Схожесть doc_1 и doc_2: -0.0573
  doc_1: deep learning uses neural networks
  doc_2: python programming for data science
Схожесть doc_1 и doc_3: 0.2031
  doc_1: deep learning uses neural networks
  doc_3: artificial intelligence is amazing
Схожесть doc_1 и doc_4: -0.2546
  doc_1: deep learning uses neural networks
  doc_4: computer vision processes images


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [54]:
# Обучаю FastText
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("FastText модель обучена :)")

try:
    similar = ft_model.wv.most_similar('мультиварка', topn=5)
    print ("Размерность: 10")
    print("Слова, похожие на 'мультиварка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'мультиварка' не найдено в словаре")
try:
    similar = ft_model.wv.most_similar('просеивать', topn=5)
    print("Слова, похожие на 'просеивать':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'просеивать' не найдено в словаре")
try:
    similar = ft_model.wv.most_similar('омлет', topn=5)
    print("Слова, похожие на 'омлет':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'омлет' не найдено в словаре")


FastText модель обучена :)
Размерность: 10
Слова, похожие на 'мультиварка':
  смешивать: 0.5659
  соус: 0.5433
  суп: 0.5221
  соль: 0.4967
  бекон: 0.4115
Слова, похожие на 'просеивать':
  взбивать: 0.7225
  смешивать: 0.6608
  соус: 0.4981
  питание: 0.4948
  торт: 0.3746
Слова, похожие на 'омлет':
  барбекю: 0.8282
  десерт: 0.5910
  паста: 0.5559
  горшок: 0.5029
  тушить: 0.4851


In [55]:
# Обучаю FastText
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("FastText модель обучена :)")

try:
    similar = ft_model.wv.most_similar('мультиварка', topn=5)
    print ("Размерность: 50")
    print("Слова, похожие на 'мультиварка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'мультиварка' не найдено в словаре")
try:
    similar = ft_model.wv.most_similar('просеивать', topn=5)
    print("Слова, похожие на 'просеивать':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'просеивать' не найдено в словаре")
try:
    similar = ft_model.wv.most_similar('омлет', topn=5)
    print("Слова, похожие на 'омлет':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'омлет' не найдено в словаре")


FastText модель обучена :)
Размерность: 50
Слова, похожие на 'мультиварка':
  печь: 0.4315
  помидоры: 0.3097
  мука: 0.2902
  яйца: 0.2593
  питание: 0.2379
Слова, похожие на 'просеивать':
  морковь: 0.2938
  чашка: 0.2609
  паста: 0.2299
  тост: 0.2263
  хлеб: 0.2163
Слова, похожие на 'омлет':
  гриль: 0.2517
  яблоки: 0.2085
  торт: 0.1937
  сахар: 0.1689
  чашка: 0.1329


In [56]:
# Обучаю FastText
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=100,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("FastText модель обучена :)")

try:
    similar = ft_model.wv.most_similar('мультиварка', topn=5)
    print ("Размерность: 100")
    print("Слова, похожие на 'мультиварка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'мультиварка' не найдено в словаре")
try:
    similar = ft_model.wv.most_similar('просеивать', topn=5)
    print("Слова, похожие на 'просеивать':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'просеивать' не найдено в словаре")
try:
    similar = ft_model.wv.most_similar('омлет', topn=5)
    print("Слова, похожие на 'омлет':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'омлет' не найдено в словаре")


FastText модель обучена :)
Размерность: 100
Слова, похожие на 'мультиварка':
  дрожжи: 0.2229
  уголь: 0.1577
  печь: 0.1538
  тушить: 0.1489
  завтрак: 0.1454
Слова, похожие на 'просеивать':
  смешивать: 0.4179
  взбивать: 0.2472
  барбекю: 0.1822
  резать: 0.1805
  салат: 0.1802
Слова, похожие на 'омлет':
  сахар: 0.2303
  тост: 0.1683
  торт: 0.1488
  яблоки: 0.1377
  духовка: 0.1161
